In [1]:
# ==========================================================
# BLOCCO 1: Caricamento dataset e scaler per training modelli
# ==========================================================
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

print("📂 Caricamento dataset e scaler...")

# 1️⃣ Carica dataset
X_train_scaled = pd.read_csv("model_data/X_train_scaled.csv")
X_test_scaled = pd.read_csv("model_data/X_test_scaled.csv")
y_train = pd.read_csv("model_data/y_train.csv").squeeze()
y_test = pd.read_csv("model_data/y_test.csv").squeeze()

print(f"✅ Dataset caricati:")
print(f"   X_train: {X_train_scaled.shape}, X_test: {X_test_scaled.shape}")

# 2️⃣ Carica scaler
scaler_latent = joblib.load("model_data/scaler_latent.pkl")
print("✅ Scaler caricato.")

# 3️⃣ (Opzionale) Carica encoder
try:
    encoder = load_model("model_data/encoder_best.keras")
    print("✅ Encoder caricato con successo.")
except Exception as e:
    print(f"⚠️ Encoder non trovato o non necessario: {e}")

print("\n🚀 Pronto per l'addestramento dei modelli!")


📂 Caricamento dataset e scaler...
✅ Dataset caricati:
   X_train: (264285, 16), X_test: (66072, 16)
✅ Scaler caricato.
✅ Encoder caricato con successo.

🚀 Pronto per l'addestramento dei modelli!


In [ ]:
# ==========================================================
# BLOCCO 7: Random Forest Classifier con GridSearchCV (griglia ridotta) e Analisi Completa
# ==========================================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_auc_score, roc_curve, precision_recall_curve, auc
)
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import time

print("🌲 Addestramento Random Forest sullo spazio latente (con GridSearchCV “smart”)...")

# 1️⃣ Calcolo pesi per classi sbilanciate
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
weights_dict = dict(zip(np.unique(y_train), class_weights))
print("\n⚖️ Pesi di bilanciamento per classe:")
for k, v in weights_dict.items():
    print(f"  Classe {k}: {v:.3f}")

# 2️⃣ Definizione griglia “smart” per accelerare il GridSearch
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

rf_clf = RandomForestClassifier(
    class_weight=weights_dict,
    n_jobs=-1,
    random_state=42
)

# 3️⃣ Cross-validation 5-fold stratificata
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=rf_clf,
    param_grid=param_grid,
    scoring='f1_weighted',  # ottimizziamo sul weighted F1
    cv=cv,
    verbose=2,
    n_jobs=-1
)

# 4️⃣ Addestramento con cronometro
start = time.time()
grid_search.fit(X_train_scaled, y_train)
elapsed = time.time() - start
print(f"\n✅ GridSearchCV completato in {elapsed/60:.2f} minuti.")

print("\n🏆 Migliori iperparametri trovati:")
print(grid_search.best_params_)
print("✅ Best weighted F1:", grid_search.best_score_)

# 5️⃣ Miglior modello
best_rf = grid_search.best_estimator_

# 6️⃣ Predizione sul test set
y_pred = best_rf.predict(X_test_scaled)
y_prob = best_rf.predict_proba(X_test_scaled)

# 7️⃣ Metriche di valutazione
print("\n📊 Report di classificazione (test set):")
print(classification_report(y_test, y_pred, digits=4))

# Specificità (True Negative Rate)
cm = confusion_matrix(y_test, y_pred)
tn = np.diag(cm)
fp = cm.sum(axis=0) - np.diag(cm)
specificity = tn / (tn + fp)
print("\n📈 Specificità media (macro): {:.4f}".format(np.mean(specificity)))

# 8️⃣ Matrice di Confusione (grafico + tabella)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# 🔹 Grafico
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax[0])
ax[0].set_title("Matrice di Confusione - Grafico")
ax[0].set_xlabel("Predetto")
ax[0].set_ylabel("Reale")

# 🔹 Tabella numerica
cm_df = pd.DataFrame(cm, 
                     index=[f"Reale {c}" for c in best_rf.classes_],
                     columns=[f"Pred {c}" for c in best_rf.classes_])
ax[1].axis("off")
ax[1].table(cellText=cm_df.values, colLabels=cm_df.columns, rowLabels=cm_df.index, loc='center')
ax[1].set_title("Matrice di Confusione - Tabella")

plt.tight_layout()
plt.show()

print("""
📘 Interpretazione:
- La diagonale rappresenta le predizioni corrette (valori alti = buone prestazioni).
- Le celle fuori diagonale indicano errori di classificazione.
- Una diagonale ben marcata significa che il modello distingue bene le classi.
""")

# 9️⃣ Curva ROC e AUC (multiclasse)
if len(np.unique(y_test)) > 2:
    from sklearn.preprocessing import label_binarize
    y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
    fpr, tpr, roc_auc = {}, {}, {}
    for i, cls in enumerate(np.unique(y_train)):
        fpr[cls], tpr[cls], _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
        roc_auc[cls] = auc(fpr[cls], tpr[cls])

    plt.figure(figsize=(8, 6))
    for cls in roc_auc.keys():
        plt.plot(fpr[cls], tpr[cls], label=f"Classe {cls} (AUC = {roc_auc[cls]:.3f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title("Curva ROC per classe")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.grid(True)
    plt.show()

    print("""
📘 Interpretazione ROC:
- Curva vicino all'angolo in alto a sinistra = migliori prestazioni.
- AUC vicino a 1.0 indica ottima capacità discriminante.
""")

# 🔟 Feature Importances (spazio latente)
importances = pd.Series(best_rf.feature_importances_, index=X_train_scaled.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=importances.values[:15], y=importances.index[:15], palette="viridis")
plt.title("Top 15 Feature Importances (Latent Features)")
plt.xlabel("Importanza")
plt.ylabel("Feature Latente")
plt.tight_layout()
plt.show()

print("""
📘 Interpretazione Feature Importances:
- Mostra quali dimensioni latenti influenzano di più le decisioni del modello.
- Valori più alti = feature più discriminanti.
- Aiuta a capire la struttura interna degli embeddings appresi.
""")

print("🏁 Addestramento e valutazione Random Forest completati con successo!")


🌲 Addestramento Random Forest sullo spazio latente (con GridSearchCV “smart”)...

⚖️ Pesi di bilanciamento per classe:
  Classe Discovery: 5.154
  Classe Other: 27.749
  Classe Reconnaissance: 1.687
  Classe Resource Development: 0.315
Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
# ==========================================================
# BLOCCO X: XGBoost Classifier con GridSearchCV e Analisi Completa
# ==========================================================
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_auc_score, roc_curve, precision_recall_curve, auc
)
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import time

print("🔥 Addestramento XGBoost sullo spazio latente (GridSearchCV “smart”)...")

# 1️⃣ Calcolo pesi per classi sbilanciate
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
weights_dict = dict(zip(np.unique(y_train), class_weights))
print("\n⚖️ Pesi di bilanciamento per classe:")
for k, v in weights_dict.items():
    print(f"  Classe {k}: {v:.3f}")

# ⚠️ Nota: per dataset molto sbilanciati, in binaria puoi usare scale_pos_weight = N_neg/N_pos
# per multi-classe è meno diretto, meglio usare class_weight come sopra

# 2️⃣ Definizione griglia “smart” per XGBoost
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 1]
}

# 3️⃣ Creazione modello base
xgb_clf = XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    use_label_encoder=False,
    n_jobs=-1,
    random_state=42
)

# 4️⃣ GridSearchCV con 5-fold Stratified
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# 5️⃣ Addestramento
start = time.time()
grid_search.fit(X_train_scaled, y_train)
elapsed = time.time() - start
print(f"\n✅ GridSearchCV XGBoost completato in {elapsed/60:.2f} minuti.")

print("\n🏆 Migliori iperparametri trovati:")
print(grid_search.best_params_)

# 6️⃣ Miglior modello e predizioni
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test_scaled)
y_prob = best_xgb.predict_proba(X_test_scaled)

# 7️⃣ Report di classificazione e specificità
print("\n📊 Report di classificazione (test set):")
print(classification_report(y_test, y_pred, digits=4))

cm = confusion_matrix(y_test, y_pred)
tn = np.diag(cm)
fp = cm.sum(axis=0) - np.diag(cm)
specificity = tn / (tn + fp)
print("\n📈 Specificità media (macro): {:.4f}".format(np.mean(specificity)))

# 8️⃣ Matrice di Confusione (grafico + tabella)
fig, ax = plt.subplots(1,2, figsize=(14,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax[0])
ax[0].set_title("Matrice di Confusione - Grafico")
ax[0].set_xlabel("Predetto")
ax[0].set_ylabel("Reale")
cm_df = pd.DataFrame(cm, index=[f"Reale {c}" for c in best_xgb.classes_],
                     columns=[f"Pred {c}" for c in best_xgb.classes_])
ax[1].axis("off")
ax[1].table(cellText=cm_df.values, colLabels=cm_df.columns, rowLabels=cm_df.index, loc='center')
ax[1].set_title("Matrice di Confusione - Tabella")
plt.tight_layout()
plt.show()

print("""
📘 Interpretazione:
- La diagonale rappresenta le predizioni corrette.
- Le celle fuori diagonale indicano errori di classificazione.
- Una diagonale marcata significa che il modello distingue bene le classi.
""")

# 9️⃣ Curva ROC e AUC (multiclasse)
y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
fpr, tpr, roc_auc = {}, {}, {}
for i, cls in enumerate(np.unique(y_train)):
    fpr[cls], tpr[cls], _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
    roc_auc[cls] = auc(fpr[cls], tpr[cls])

plt.figure(figsize=(8,6))
for cls in roc_auc.keys():
    plt.plot(fpr[cls], tpr[cls], label=f"Classe {cls} (AUC = {roc_auc[cls]:.3f})")
plt.plot([0,1],[0,1],'k--')
plt.title("Curva ROC per classe")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid(True)
plt.show()

print("""
📘 Interpretazione ROC:
- Curva vicina all'angolo in alto a sinistra = ottime prestazioni.
- AUC vicino a 1.0 = migliore discriminazione della classe.
""")

# 🔟 Curva Precision-Recall (multiclasse)
plt.figure(figsize=(8,6))
for i, cls in enumerate(np.unique(y_train)):
    precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_prob[:, i])
    plt.plot(recall, precision, label=f"Classe {cls}")
plt.title("Curva Precision-Recall")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.grid(True)
plt.show()

print("""
📘 Interpretazione Precision-Recall:
- Precision = purezza predizioni positive.
- Recall = capacità di catturare veri positivi.
- Curva alta a destra = buon equilibrio precision/recall.
""")

# 11️⃣ Feature Importances
importances = pd.Series(best_xgb.feature_importances_, index=X_train_scaled.columns).sort_values(ascending=False)
plt.figure(figsize=(10,6))
sns.barplot(x=importances.values[:15], y=importances.index[:15], palette="viridis")
plt.title("Top 15 Feature Importances (Latent Features)")
plt.xlabel("Importanza")
plt.ylabel("Feature Latente")
plt.tight_layout()
plt.show()

print("""
📘 Interpretazione Feature Importances:
- Mostra quali dimensioni latenti influenzano di più le decisioni.
- Valori più alti = feature più discriminanti.
- Aiuta a capire la struttura interna degli embeddings appresi.
""")

print("🏁 Addestramento e valutazione XGBoost completati con successo!")
